### 랭체인 준비

In [ ]:
# 패키지 설치
!pip install langchain==0.3.7
!pip install langchain-google-genai
!pip install langchain_community
!pip install langgraph

In [ ]:
from google.colab import userdata
import os

# 환경 변수 준비(좌측 상단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")

###랭스미스

In [ ]:
import os
from uuid import uuid4

# 환경 변수 준비
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

### LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [ ]:
# LLM 실행
messages = [
    ("human", "컴퓨터 게임을 만드는 새로운 회사 이름을 하나만 제안해주세요.")
]
output = llm.invoke(messages)
print(type(output))
print(output)

### 프롬프트 템플릿

In [ ]:
from langchain.prompts import ChatPromptTemplate

# 프롬프트 템플릿 준비
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{product}을 만드는 새로운 회사 이름을 하나만 제안해주세요."),
])

In [ ]:
# 프롬프트 템플릿 실행
output = prompt_template.invoke({"product": "가정용 로봇"})
print(type(output))
print(output)

### 출력 파서

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 출력 파서 준비
output_parser = StrOutputParser()

In [ ]:
from langchain.schema import AIMessage

# 출력 파서 실행
message = AIMessage(content="AI로부터의 메시지입니다.")
output = output_parser.invoke(message)
print(type(output))
print(output)

### 체인

In [ ]:
# 체인 준비
chain = prompt_template | llm | output_parser

In [ ]:
# 체인 실행
output = chain.invoke({"product": "가정용 로봇"})
print(type(output))
print(output)

### 에이전트

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(x: float, y: float) -> float:
    """'x'와 'y'를 곱함"""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """'x'를 'y'로 거듭제곱"""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """'x'와 'y'를 더함"""
    return x + y

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [ ]:
# httpx 패키지 설치
!pip install httpx

In [ ]:
from langgraph.prebuilt import chat_agent_executor

# 에이전트 준비
agent_executor = chat_agent_executor.create_tool_calling_executor(
    llm,
    tools=[multiply, exponentiate, add]
)

In [ ]:
# 도구를 사용하지 않는 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "안녕하세요!")]}
)
response["messages"][-1].content

In [ ]:
# 도구를 사용한 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "1에 2를 더하면?")]}
)
response["messages"][-1].content

In [ ]:
# 중간 처리 확인
for message in response["messages"]:
    print(message)

In [ ]:
# 도구를 두 번 사용한 질의 응답
response = agent_executor.invoke(
    {"messages": [("human", "(1+2)*3은?")]}
)
response["messages"][-1].content

In [ ]:
# 중간 처리 확인
for message in response["messages"]:
    print(message)